In [ ]:
# Databricks notebook: Download artifacts for latest run of a given task of THIS job
# ---------------------------------------------------------------------------------

# PARAMETERS
# ----------
dbutils.widgets.text("TASK_KEY", "")               # task_key (task name) within this job
dbutils.widgets.text("SOURCE_PATH", "")            # subfolder inside artifacts, empty for root
dbutils.widgets.text("DEST_PATH", "")   # local/DBFS dest on this cluster

In [ ]:
TASK_KEY = dbutils.widgets.get("TASK_KEY").strip()
SOURCE_PATH = dbutils.widgets.get("SOURCE_PATH").strip()
DEST_PATH = dbutils.widgets.get("DEST_PATH").strip()

In [ ]:
if not TASK_KEY:
    raise ValueError("TASK_KEY widget must be provided (task_key within the job).")

In [ ]:
import dbruntime.databricks_repl_context as repl_ctx
import subprocess
from pathlib import Path


In [ ]:
def run_cmd(cmd: str) -> str:
    """Run a shell command and return stdout as string. Raises on non-zero exit."""
    print(f"Executing: {cmd}")
    result = subprocess.run(
        cmd,
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
    )
    if result.returncode != 0:
        raise RuntimeError(
            f"Command failed ({result.returncode}).\n"
            f"STDOUT:\n{result.stdout}\n\nSTDERR:\n{result.stderr}"
        )
    return result.stdout

In [ ]:
def ensure_dir(path: str):
    Path(path).mkdir(parents=True, exist_ok=True)

In [ ]:
# 1. Get JOB_ID of *this* notebook's job
# --------------------------------------
JOB_ID = dbutils.notebook().getContext().toJson()['job_id']
print(f"Current job id (JOB_ID) = {JOB_ID}")

In [ ]:
# 2. Ensure Jobs API 2.1 for CLI
# ------------------------------
run_cmd("databricks jobs configure --version=2.1")